In [86]:
from tabula import read_pdf
import os
import pandas as pd
import pickle
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



def process_one_table(one_table):
    first_row = one_table.iloc[0, ].astype(str).str.upper()
    try:
        case_no_index = [i for i, e in enumerate(first_row) if re.search("CASE NUM", e,
                                                                        re.IGNORECASE)][0]
    except:
        case_no_index = 0
    try:
        home_school_index = [i for i, e in enumerate(first_row) if re.search("STUDENT HOM", e,
                                                                            re.IGNORECASE)][0]
    except:
        home_school_index = 0
    try:
        attending_school_index  = [i for i, e in enumerate(first_row) if re.search("STUDENT ATTEN", e,
                                                                                  re.IGNORECASE)][0]
    except:
        attending_school_index = 0
    try:
        casetype_index  = first_row.tolist().index("CASE TYPE")
    except:
        casetype_index = 0
    try:
        dcps_index = [i for i, e in enumerate(first_row) if re.search("AGAINST DCP", e,
                                                                     re.IGNORECASE)][0]
    except:
        dcps_index = 0
    try:
        charter_index = [i for i, e in enumerate(first_row) if re.search("AGAINST Charter", e,
                                                                        re.IGNORECASE)][0]
    except:
        charter_index = 0
    try:
        osse_index = [i for i, e in enumerate(first_row) if re.search("OSSE", e,
                                                                     re.IGNORECASE)][0]
    except:
        osse_index = 0

    ## construct new df
    date_cols = one_table.iloc[1:, -2:]
    date_cols.columns = ['date_order_1', "date_order_2"]
    #return(date_cols)
    new_df = pd.concat([pd.DataFrame({'case_no': one_table.iloc[1:, case_no_index],
                          'home_school': one_table.iloc[1:, home_school_index],
                          'attending_school': one_table.iloc[1:, attending_school_index],
                          'casetype': one_table.iloc[1:, casetype_index],
                          'dcps_school_against': one_table.iloc[1:, dcps_index],
                          'charter_school_against': one_table.iloc[1:, charter_index],
                          'osse_against': one_table.iloc[1:, osse_index]}),
                       date_cols],
                      axis = 1)

    return(new_df)







In [88]:
############### Read raw tables
os.getcwd()
dc_filings = pickle.load(open("../../raw_input/dc/dc_rawtables.p", "rb"))

'/Users/rebeccajohnson/Dropbox/rights_towhat_chapter/code/dc_filings'

In [89]:

## process the tables
print("starting processing")
processed_filings = [process_one_table(filing) for filing in dc_filings]
print("ending processing")
processed_all = pd.concat(processed_filings)

starting processing
ending processing


In [90]:
## instead of padding, use following decision rule:
## (1) take row with case number
## (2) see how many missing
processed_complete = processed_all[~processed_all.case_no.isnull()].drop_duplicates()


In [91]:

## write 
processed_complete.to_csv("../../intermediate_objects/processed_filings_preclean_dc.csv", index = False)